# Notebook figuring out the code for getting most shared articles and NLP including Unit Testing

In [60]:
# import libraries
import pymongo
import requests
from pymongo import MongoClient
from datetime import datetime, timedelta
import pandas as pd
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import nltk

In [2]:
# start mongo connection
client = MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [32]:
# database
db = client.nytimes
# collection
fb = db.facebook
fb.count_documents({})

21

In [34]:
list(fb.find({}))

[{'_id': ObjectId('60cc0549040bcf90fd0cc356'),
  'uri': 'nyt://article/668b76e2-5702-552e-b68b-0e02ff90f40e',
  'url': 'https://www.nytimes.com/2021/06/12/world/china-covid-delta-variant-guangzhou.html',
  'id': 100000007810026,
  'asset_id': 100000007810026,
  'source': 'New York Times',
  'published_date': '2021-06-12',
  'updated': '2021-06-15 07:05:09',
  'section': 'World',
  'subsection': '',
  'nytdsection': 'world',
  'adx_keywords': 'internal-essential;Coronavirus (2019-nCoV);Fever;Guangzhou (China);China',
  'column': None,
  'byline': 'By Keith Bradsher',
  'type': 'Article',
  'title': 'In China’s latest outbreak, doctors say the infected get sicker, faster.',
  'abstract': 'Four-fifths of symptomatic cases develop fevers, more than in earlier outbreaks. The findings match trends in other countries where the Delta variant is spreading.',
  'des_facet': ['internal-essential', 'Coronavirus (2019-nCoV)', 'Fever'],
  'org_facet': [],
  'per_facet': [],
  'geo_facet': ['Guangzho

In [38]:
# function to add most shared articles on FB into database collection
def fb_shared():
    response=requests.get('https://api.nytimes.com/svc/mostpopular/v2/shared/7/facebook.json?api-key=kn9QbtAA4N9fdVC9NAhjC1NyDTvF8hB3').json()
    data = response['results']
    fb.insert_many(data)

In [39]:
fb_shared()

In [40]:
fb.count_documents({})

41

In [41]:
fb.update_many(
  {},
  [{ '$set': { 'updated': { '$toDate': "$updated" } } }]
)

In [42]:
list(fb.find({}))

[{'_id': ObjectId('60cc0549040bcf90fd0cc356'),
  'uri': 'nyt://article/668b76e2-5702-552e-b68b-0e02ff90f40e',
  'url': 'https://www.nytimes.com/2021/06/12/world/china-covid-delta-variant-guangzhou.html',
  'id': 100000007810026,
  'asset_id': 100000007810026,
  'source': 'New York Times',
  'published_date': '2021-06-12',
  'updated': datetime.datetime(2021, 6, 15, 7, 5, 9),
  'section': 'World',
  'subsection': '',
  'nytdsection': 'world',
  'adx_keywords': 'internal-essential;Coronavirus (2019-nCoV);Fever;Guangzhou (China);China',
  'column': None,
  'byline': 'By Keith Bradsher',
  'type': 'Article',
  'title': 'In China’s latest outbreak, doctors say the infected get sicker, faster.',
  'abstract': 'Four-fifths of symptomatic cases develop fevers, more than in earlier outbreaks. The findings match trends in other countries where the Delta variant is spreading.',
  'des_facet': ['internal-essential', 'Coronavirus (2019-nCoV)', 'Fever'],
  'org_facet': [],
  'per_facet': [],
  'geo_

In [45]:
# function to call api and insert documents into database, change date to datetime object, and return list of articles 
def fb_shared():
    response=requests.get('https://api.nytimes.com/svc/mostpopular/v2/shared/7/facebook.json?api-key=kn9QbtAA4N9fdVC9NAhjC1NyDTvF8hB3').json()
    data = response['results']
    shared.insert_many(data)
    shared.update_many({},[{ '$set': { 'updated': { '$toDate': "$updated" } } }])
    seven_days_ago = datetime.today() - timedelta(days=6)
    seven_days_ago = seven_days_ago.replace(hour=0, minute=0, second=0, microsecond=0)
    tomorrow = datetime.today() + timedelta(days=1)
    tomorrow = tomorrow.replace(hour=0, minute=0, second=0, microsecond=0)
    extraction = list(shared.find({'updated': {'$lt': tomorrow, '$gte': seven_days_ago}}, {'_id': 0, 'abstract':1, 'published_date':1, 'title':1,
                                                                                 'adx_keywords':1, 'byline':1, 'section':1}))
    return extraction

In [46]:
# run fb_shared function
fb_shared()
shared.count_documents({})

20

In [47]:
# change date field to datetime
shared.update_many(
  {},
  [{ '$set': { 'updated': { '$toDate': "$updated" } } }]
)

In [51]:
list(shared.find({}))

[{'_id': ObjectId('60d23d0bf084f5fec4376a19'),
  'uri': 'nyt://article/d46a3df0-8a9f-521e-b466-74cec07f22d3',
  'url': 'https://www.nytimes.com/2021/06/21/sports/football/carl-nassib-gay-nfl.html',
  'id': 100000007827346,
  'asset_id': 100000007827346,
  'source': 'New York Times',
  'published_date': '2021-06-21',
  'updated': datetime.datetime(2021, 6, 22, 15, 22, 20),
  'section': 'Sports',
  'subsection': 'N.F.L.',
  'nytdsection': 'sports',
  'adx_keywords': 'Football;Homosexuality and Bisexuality;Nassib, Carl (1993- );National Football League;Las Vegas Raiders',
  'column': None,
  'byline': 'By Ken Belson',
  'type': 'Article',
  'title': 'Raiders’ Carl Nassib Announces He’s Gay, an N.F.L. First',
  'abstract': 'The Raiders defensive lineman came out in a statement posted to his Instagram account on Monday, becoming the first active player in the league to publicly identify as gay.',
  'des_facet': ['Football', 'Homosexuality and Bisexuality'],
  'org_facet': ['National Footbal

In [53]:
# extract the articles from database
extraction = list(shared.find({'updated': {'$lt': tomorrow, '$gte': seven_days_ago}}, {'_id': 0, 'abstract':1, 'published_date':1, 'title':1,
                                                                                 'adx_keywords':1, 'byline':1, 'section':1}))
len(extraction)

13

In [54]:
list(extraction)

[{'published_date': '2021-06-21',
  'section': 'Sports',
  'adx_keywords': 'Football;Homosexuality and Bisexuality;Nassib, Carl (1993- );National Football League;Las Vegas Raiders',
  'byline': 'By Ken Belson',
  'title': 'Raiders’ Carl Nassib Announces He’s Gay, an N.F.L. First',
  'abstract': 'The Raiders defensive lineman came out in a statement posted to his Instagram account on Monday, becoming the first active player in the league to publicly identify as gay.'},
 {'published_date': '2021-06-21',
  'section': 'Business',
  'adx_keywords': 'United States Economy;Consumer Price Index;Prices (Fares, Fees and Rates);Inflation (Economics);Wood and Wood Products;Stimulus (Economic);Labor and Jobs;Federal Reserve System',
  'byline': 'By Matt Phillips',
  'title': 'As Lumber Prices Fall, the Threat of Inflation Loses Its Bite',
  'abstract': 'Costs soared partly because of do-it-yourselfers’ spending stimulus checks, but a month of declines show that consumers aren’t about to trigger run

In [70]:
# function for transforming articles into dataframe and text preprocessing

def transformation(extraction, column):
    df = pd.DataFrame(extraction)
    alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower())
    df['column'] = df.abstract.map(alphanumeric).map(punc_lower)
    return df['column']

In [72]:
# convert extraction into dataframe and text preprocessing
clean_abstract = transformation(extraction)

In [91]:
clean_abstract

0     the raiders defensive lineman came out in a st...
1     costs soared partly because of doityourselfers...
2     a vote by the us bishops was the most dramatic...
3     the storm was blamed for the deaths of at leas...
4     the association argued that the payments were ...
5     confidential court records obtained by the tim...
6     a professional athlete raping a reporter is a ...
7     a fractured coalition of justices limited the ...
8     his platform on fox news made him a big player...
9     one in five in that age group mostly in the so...
10    sang ho baek   “had been battling injuries thr...
11    the accident and a subsequent coverup have ren...
12    with new opportunities and a different perspec...
Name: abstract, dtype: object

In [73]:
# create stopwords list
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['much', 'people', 'one', 'thing', 'always', 'every', 'everyone', 'yes', 'know', 'years', 'stop', 'let', 'need', 
               'someething', 'find', 'others', 'enough', 'seems', 'often', 'never', 'still', 'like', 'say', 'hope', 'small', 
               'almost', 'take', 'important', 'said', 'turned', 'making', 'like', 'also', 'need', 'get', 'way', 'got', 'came', 'would',
               'could']
stopwords.extend(newStopWords)

In [74]:
# function to perform tfidf
def tfidf(docs, stopwords):
    tf = TfidfVectorizer(stop_words=stopwords)
    word_matrix = tf.fit_transform(docs)
    vocab = tf.get_feature_names()
    return word_matrix, vocab

word_matrix, vocab = tfidf(clean_abstract, stopwords)
nmf=NMF(n_components=10)
fitted_nmf=nmf.fit(word_matrix)

C:\Users\sandy\anaconda3_2nd_download\envs\metis\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [75]:
# function to display the topics
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [77]:
# 10 topics
display_topics(nmf, vocab, 10)


Topic  0
trump, fox, throughout, example, since

Topic  1
role, effect, fractured, coalition, executive

Topic  2
lineman, first, publicly, statement, posted

Topic  3
checks, partly, show, increases, soared

Topic  4
group, received, increasing, five, outbreaks

Topic  5
accident, subsequent, origins, relevance, renewed

Topic  6
story, men, raping, professional, power

Topic  7
payments, violate, barring, association, argued

Topic  8
leave, jobs, perspective, different, pandemic

Topic  9
least, tornadoes, children, spawned, storm


In [90]:
fb.count_documents({})

41

In [94]:
# function to add articles to database collection for unit testing
def fb_shared():
    response=requests.get('https://api.nytimes.com/svc/mostpopular/v2/shared/7/facebook.json?api-key=kn9QbtAA4N9fdVC9NAhjC1NyDTvF8hB3')
    response_json = response.json()
    data = response_json['results']
    fb.insert_many(data)
    return response

In [99]:
# function for turning extraction into Dataframe and text preprocessing for unit testing
def transformation(extraction, column):
    df = pd.DataFrame(extraction)
    alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower())
    df['abstract'] = df.abstract.map(alphanumeric).map(punc_lower)
    return df['abstract']

In [100]:
transformation(extraction)

0     the raiders defensive lineman came out in a st...
1     costs soared partly because of doityourselfers...
2     a vote by the us bishops was the most dramatic...
3     the storm was blamed for the deaths of at leas...
4     the association argued that the payments were ...
5     confidential court records obtained by the tim...
6     a professional athlete raping a reporter is a ...
7     a fractured coalition of justices limited the ...
8     his platform on fox news made him a big player...
9     one in five in that age group mostly in the so...
10    sang ho baek   “had been battling injuries thr...
11    the accident and a subsequent coverup have ren...
12    with new opportunities and a different perspec...
Name: abstract, dtype: object

In [101]:
# using column instead of specific 'abstract' column in function
def transformation(extraction, column):
    df = pd.DataFrame(extraction)
    alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower())
    df['column'] = df.abstract.map(alphanumeric).map(punc_lower)
    return df['column']

In [104]:
transformation(extraction, 'abstract')

0     the raiders defensive lineman came out in a st...
1     costs soared partly because of doityourselfers...
2     a vote by the us bishops was the most dramatic...
3     the storm was blamed for the deaths of at leas...
4     the association argued that the payments were ...
5     confidential court records obtained by the tim...
6     a professional athlete raping a reporter is a ...
7     a fractured coalition of justices limited the ...
8     his platform on fox news made him a big player...
9     one in five in that age group mostly in the so...
10    sang ho baek   “had been battling injuries thr...
11    the accident and a subsequent coverup have ren...
12    with new opportunities and a different perspec...
Name: column, dtype: object

In [105]:
import unittest

In [115]:
class TestMethods(unittest.TestCase):
    def test_presence_abstract_field(self):
        self.assertIsNotNone(df['abstract'], "No abstract field")
        return "Passed!"

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
